In [10]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import open3d as o3d
import os

In [11]:
# Utility Functions

def display_image(img):
    img2 = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img2)
    plt.show()

def draw_line(img, A, B):
    cv2.line(img, A, B, (0, 255, 0), thickness=3, lineType=8)

def add_column(original, newcol):
    return np.concatenate([original, newcol], axis=1)

def add_row(original, newrow): # didn't use
    return np.concatenate([original, newrow], axis=0)

def camera_project(P, world_point):
    # Convert the points in 3d space to homogeneous coordinates by appending 1, and making it a column vector
    homo_point = np.transpose(np.array([world_point[0], world_point[1], world_point[2], 1]))

    #Projecting the points into 2d by applying the camera projection matrix
    val = np.matmul(P, homo_point)

    # Returned pixel coordinates, homogenous(a, b, c) => (a/c, b/c) in pixels.
    # int as pixels are integral.
    return (int(val[0] / val[2]), int(val[1] / val[2]))

def draw_point(image, point, color):
    if color == "green":
        cv2.circle(image, point, 10, (0, 255, 0), thickness=3, lineType=8)
    elif color == "red":
        cv2.circle(image, point, 8, (0, 0, 255), thickness=3, lineType=8)
        
def create_output(vertices, colors, filename): # taken from github
    colors = colors.reshape(-1,3)
    vertices = np.hstack([vertices.reshape(-1,3),colors])

    ply_header = '''ply
        format ascii 1.0
        element vertex %(vert_num)d
        property float x
        property float y
        property float z
        property uchar red
        property uchar green
        property uchar blue
        end_header
        '''
    with open(filename, 'w') as f:
        f.write(ply_header %dict(vert_num=len(vertices)))
        np.savetxt(f,vertices,'%f %f %f %d %d %d')

In [12]:
lst = os.listdir('./data/img2')
# Adjust start and end to change number of pictures and which pictures to select. Ensure start < end
start = 0
end = 3
imgs = sorted(lst)[start:end]
print(imgs)

['0000000460.png', '0000000461.png', '0000000462.png']


In [14]:
pose = np.array([
    np.array([[-9.098548e-01,5.445376e-02,-4.113381e-01,-1.872835e+02],[4.117828e-02,9.983072e-01,4.107410e-02,1.870218e+00],[4.128785e-01,2.043327e-02,-9.105569e-01,5.417085e+01]]),
    np.array([[-9.283201e-01,4.509830e-02,-3.690366e-01,-1.874227e+02],[3.357747e-02,9.987291e-01,3.758534e-02,1.883289e+00],[3.702626e-01,2.249990e-02,-9.286546e-01,5.369416e+01]]),
    np.array([[-9.452512e-01,3.706129e-02,-3.242324e-01,-1.875256e+02],[2.776133e-02,9.990610e-01,3.326342e-02,1.887417e+00],[3.251607e-01,2.244117e-02,-9.453925e-01,5.321309e+01]]),
    np.array([[-9.602059e-01,3.410654e-02,-2.772028e-01,-1.876307e+02],[2.674480e-02,9.991831e-01,3.029615e-02,1.893359e+00],[2.780097e-01,2.167680e-02,-9.603337e-01,5.273710e+01]]),
    np.array([[-9.729422e-01,3.276894e-02,-2.287129e-01,-1.876915e+02],[2.667847e-02,9.992036e-01,2.967147e-02,1.900316e+00],[2.295031e-01,2.276691e-02,-9.730416e-01,5.225276e+01]]),
    np.array([[-9.831529e-01,3.240420e-02,-1.798899e-01,-1.877485e+02],[2.738709e-02,9.991654e-01,3.030454e-02,1.901338e+00],[1.807218e-01,2.486734e-02,-9.832198e-01,5.177232e+01]]),
    np.array([[-9.911510e-01,2.523874e-02,-1.303180e-01,-1.877723e+02],[2.111370e-02,9.992343e-01,3.293915e-02,1.900778e+00],[1.310496e-01,2.989617e-02,-9.909249e-01,5.129098e+01]]),
    np.array([[-9.966637e-01,1.446740e-02,-8.032513e-02,-1.877414e+02],[1.146235e-02,9.992216e-01,3.774707e-02,1.901606e+00],[8.080871e-02,3.670042e-02,-9.960537e-01,5.080341e+01]]),
    np.array([[-9.995040e-01,1.006074e-02,-2.984320e-02,-1.877165e+02],[8.815319e-03,9.990965e-01,4.157437e-02,1.903912e+00],[3.023451e-02,4.129067e-02,-9.986896e-01,5.033042e+01]]),
    np.array([[-9.996705e-01,1.307490e-02,2.209183e-02,-1.876828e+02],[1.395603e-02,9.990937e-01,4.021250e-02,1.909578e+00],[-2.154603e-02,4.050756e-02,-9.989469e-01,4.986716e+01]]),
    np.array([[-9.970705e-01,2.051987e-02,7.368440e-02,-1.876045e+02],[2.335961e-02,9.990090e-01,3.788635e-02,1.913879e+00],[-7.283396e-02,3.949659e-02,-9.965617e-01,4.940375e+01]]),
    np.array([[-9.919546e-01,2.316199e-02,1.244574e-01,-1.875198e+02],[2.767742e-02,9.990153e-01,3.467489e-02,1.916071e+00],[-1.235318e-01,3.784057e-02,-9.916189e-01,4.895472e+01]]),
    np.array([[-9.843986e-01,2.362611e-02,1.743596e-01,-1.873856e+02],[2.975169e-02,9.990255e-01,3.260177e-02,1.916040e+00],[-1.734194e-01,3.728062e-02,-9.841422e-01,4.850872e+01]]),
    np.array([[-9.745046e-01,2.207656e-02,2.232787e-01,-1.872454e+02],[2.939030e-02,9.991330e-01,2.948581e-02,1.919502e+00],[-2.224342e-01,3.529628e-02,-9.743086e-01,4.808322e+01]]),
    np.array([[-9.620042e-01,2.331542e-02,2.720374e-01,-1.870919e+02],[3.162970e-02,9.991557e-01,2.621754e-02,1.921924e+00],[-2.711965e-01,3.382584e-02,-9.619295e-01,4.766758e+01]]),
    np.array([[-9.466844e-01,2.747540e-02,3.209888e-01,-1.869050e+02],[3.735473e-02,9.989978e-01,2.465901e-02,1.925132e+00],[-3.199896e-01,3.533474e-02,-9.467619e-01,4.724980e+01]]),
    np.array([[-9.303521e-01,3.021656e-02,3.654203e-01,-1.867142e+02],[4.116827e-02,9.989052e-01,2.221412e-02,1.922721e+00],[-3.643491e-01,3.571067e-02,-9.305775e-01,4.684089e+01]]),
    np.array([[-9.132370e-01,2.475193e-02,4.066762e-01,-1.864782e+02],[3.739334e-02,9.990320e-01,2.316585e-02,1.917552e+00],[-4.057092e-01,3.636289e-02,-9.132786e-01,4.643056e+01]]),
    np.array([[-8.953418e-01,1.994046e-02,4.449332e-01,-1.862433e+02],[3.564541e-02,9.990008e-01,2.695748e-02,1.910636e+00],[-4.439511e-01,3.999598e-02,-8.951580e-01,4.602498e+01]]),
    np.array([[-8.776813e-01,1.983338e-02,4.788341e-01,-1.859999e+02],[3.901401e-02,9.987839e-01,3.014110e-02,1.906858e+00],[-4.776541e-01,4.513551e-02,-8.773878e-01,4.562057e+01]]),
    np.array([[-8.608718e-01,2.164481e-02,5.083614e-01,-1.857437e+02],[4.407236e-02,9.985119e-01,3.211896e-02,1.904895e+00],[-5.069097e-01,5.005498e-02,-8.605447e-01,4.521752e+01]])
])
len(pose)

21

In [15]:
def make_new_pt(pt_3d, C):
    P = np.array([
        [C[0][0], C[0][1], C[0][2], C[0][3]],
        [C[1][0], C[1][1], C[1][2], C[1][3]],
        [C[2][0], C[2][1], C[2][2], C[2][3]],
    ])
    pt = np.mat([pt_3d[0], pt_3d[1], pt_3d[2], 1]).T
    mat = np.matmul(P, pt)
    return mat

combination = []
A0 = np.vstack([pose[0], np.array([0, 0, 0, 1])])
combination.append(A0)
for c in range(1, len(pose)):
    A = np.vstack([pose[c], np.array([0, 0, 0, 1])])
    combination.append(A)

final_output = []
final_color = []
output_colors = []
output_points = []
Height = 0
Width = 0
disparity_map = 0
Pixel_Point_Map = {}
def main():
    global final_output, final_color, output_colors, output_points, disparity_map
    focal_length = 7.070912e+02
    k = np.float32([[7.070912e+02, 0.000000e+00, 6.018873e+02], 
         [0.000000e+00, 7.070912e+02, 1.831104e+02], 
         [0.000000e+00, 0.000000e+00, 1.000000e+00]])
    baseline = 0.53790448812

    for count, img in enumerate(imgs):
        print('./img2/' + img, './img3/' + img)
        iml = cv2.imread('./data/img2/' + img)
        imr = cv2.imread('./data/img3/' + img)
        window_size = 3
        min_disp = 16
        num_disp = 112-min_disp
        stereo = cv2.StereoSGBM_create(minDisparity = min_disp,
           numDisparities = num_disp+(2) * 16,
           blockSize = 7,
           P1 = 8*3*window_size**2,
           P2 = 32*3*window_size**2,
           disp12MaxDiff = 1,
           uniquenessRatio = 12,
           speckleWindowSize = 400,
           speckleRange = 5
       )
        disparity_map = stereo.compute(iml, imr).astype(np.float32) / 16.0
        h, w = iml.shape[:2]
        Height = h
        Width = w
        Q = np.float32([[1, 0, 0, -0.5*w],
                        [0,-1, 0,  0.5*h], 
                        [0, 0, 0, -focal_length], 
                        [0, 0, -1/baseline,  0]])
        points = cv2.reprojectImageTo3D(disparity_map, Q)
        colors = cv2.cvtColor(iml, cv2.COLOR_BGR2RGB)
        
# Hand coding reprojectImageto3D....results weren't as good as cv2 functions
# Overwriting colors and points in this process with own implementation
# 
# 
        Pixel_Point_Map[count+start] = []
        for i in range(len(iml)):
            for j in range(len(iml[i])):
                colors[i][j] = iml[i][j]
                pt_temp = Q @ np.array([i, j, disparity_map[i][j]/16.0, 1]).T
                W = pt_temp[3]
                to_ap = [pt_temp[0]/W, pt_temp[1]/W, pt_temp[2]/W]
                Pixel_Point_Map[count+start].append(([i, j], to_ap))
                points[i][j] = to_ap

        print(points.shape)
        print(colors.shape)
        mask = disparity_map > disparity_map.min()
        output_points = points[mask]
        output_colors = colors[mask]
        for i,pt in enumerate(output_points):
            final_output.append(np.array(make_new_pt(pt, combination[count+start]).T).astype('float32')[0])
            final_color.append(output_colors[i])
        print("Length of output Points:", len(output_points), "count:", count)
    final_output = np.array(final_output)
    final_color = np.array(final_color)
    print(final_output.shape, final_color.shape)
    create_output(final_output, final_color, 'jnrecon_mult.ply')
main()

./img2/0000000460.png ./img3/0000000460.png
(370, 1226, 3)
(370, 1226, 3)
Length of output Points: 294374 count: 0
./img2/0000000461.png ./img3/0000000461.png
(370, 1226, 3)
(370, 1226, 3)
Length of output Points: 295340 count: 1
./img2/0000000462.png ./img3/0000000462.png
(370, 1226, 3)
(370, 1226, 3)
Length of output Points: 281749 count: 2
(871463, 3) (871463, 3)


In [16]:
# Pixel_Point_Map
# with open('pixel-point-map.txt', 'w') as f:
#     print(Pixel_Point_Map[0], file=f) 
# Dictionary where keys correspond to which image pair we're taking
# value is a list of tuples.
# Each tuple first element is a list [x, y] of the pixel coordinates
# Second element is a list [X, Y, Z] of the 3d world coordinates
# Form of the Dictionary:
# {0: [([x_p1, y_p1], [X_w1, Y_w1, Z_w1]), ([x_p2, y_p2], [X_w2, Y_w2, Z_w2]), ...]
#  1: []...}

In [17]:
# No need to use the code below
# p3p part
#d = (f - Z' * b ) / ( Z' * a)

#Ix = X' * ( d * a + b ) + Cx

#Iy = Y' * ( d * a + b ) + Cy
# Height = 370
# Width = 1226
# def find_impts(X, Y, Z):
#     f = 7.070912e+02
#     baseline = 0.53790448812
#     a = -1/baseline
#     b = 0
#     d = (f - Z * b) / (Z * a)
#     Ix = X * (d*a + b) + 0.5*Width
#     Iy = Y * (d*a + b) + 0.5*Height
#     return (int(Iy), int(Ix))
# inv_c = []
# hold_pts = []
# for c in combination:
#     inv_c.append(np.linalg.inv(c))
# for i, pt in enumerate(final_output):
#     pose_num = int(i/304696)
#     temp_pt = np.array([pt[0], pt[1], pt[2], 1]).T
#     out_pt = np.matmul(inv_c[pose_num], temp_pt)
#     hold_pts.append(find_impts(out_pt[0], out_pt[1], out_pt[2]))

# Part 2: Recovering the Pose

In [53]:
# Estimated R|t matrix 
Q = np.array([ [-9.05e-01, 5.5e-02, -4.2e-01, -1.9e+02], [4.21e-02, 9.89e-01, 4.23e-02, 1.760218e+00], [4.198785e-01, 2.243327e-02, -9.115569e-01, 5.517085e+01]])

# Calibration Matrix
cal_mat = [[7.070912e+02, 0.000000e+00, 6.018873e+02], 
           [0.000000e+00, 7.070912e+02, 1.831104e+02],
           [0.000000e+00, 0.000000e+00, 1.000000e+00] ]
cal_mat = np.array(cal_mat)

# Initial estimate of P 
P_est = np.dot(cal_mat, Q) # 3x4 matrix 

pts_3d = [] # 3D points from the point cloud 
pts_2d = [] # their corresponding 2D location estimated based on our P matrix 
actual_pts_2d = [] # actual 2D pixel values 

for i in range(0, 10000, 10): #taking random 100 points and storing them in arrays
    pt_3d = Pixel_Point_Map[0][i][1]
    pt_3d = np.array(pt_3d)
    pt_3d = np.append(pt_3d, 1)
    
    pts_3d.append(pt_3d)
    
    actual_pt_2d = Pixel_Point_Map[0][i][0]
    actual_pt_2d = np.append(actual_pt_2d, 1)
    actual_pts_2d.append(actual_pt_2d)
    
pts_3d = np.array(pts_3d)
actual_pts_2d = np.array(actual_pts_2d)

# Now running Gauss Newton algorithm. With every iteration, we try to find a better estimate for P
print(pts_3d[0][3])
for i in range(20): # number of iterations to be updated later
    pts_2d = []
    for pt_3d in pts_3d:
        pt_2d = np.dot(P_est, pt_3d)
        pt_2d = pt_2d/pt_2d[2]
        pts_2d.append(pt_2d)

    error = pts_2d - actual_pts_2d # 1000x3
    error = np.delete(error, 2, axis=1) # 1000x2 i.e. 1000 points and 2 coordinates for each 

    sum_sq_error = 0
    for obj in error:
        sq_error = np.dot(obj.T, obj)
        sum_sq_error = sum_sq_error + sq_error

#     print(sum_sq_error)
#     print(P_est[0][0])
    #calculating J
    for pt_3d in pts_3d:
        numerator = P_est[0][0]*pt_3d[0] + P_est[0][1]*pt_3d[1] + P_est[0][2]*pt_3d[2] + P_est[0][3]
        denom = P_est[2][0]*pt_3d[0] + P_est[2][1]*pt_3d[1] + P_est[2][2]*pt_3d[2] + P_est[2][3]
        print(denom)
        
        





1.0
-169.3539137873113
-169.22519958525294
-169.09648538319456
-168.9677711811362
-168.8390569790778
-168.71034277701943
-168.58162857496106
-168.45291437290268
-168.3242001708443
-168.19548596878593
-168.06677176672756
-167.93805756466918
-167.8093433626108
-167.68062916055243
-167.55191495849405
-167.42320075643568
-167.2944865543773
-167.16577235231892
-167.03705815026055
-166.9083439482022
-114.47282878686147
-166.65091554408545
-138.30672208031447
-119.17485791341798
-121.97764835032822
-118.40319606733485
-152.18370737543142
-152.0630378110017
-139.7592377614795
-128.82252660634867
-165.49248772556007
-106.19594770012313
-151.45968998885309
-165.10634511938494
-164.97763091732656
-164.8489167152682
-119.00028070353252
-164.59148831115144
-96.73713209274442
-109.45783243027603
-101.99418572893822
-102.37225048113865
-163.94791730085956
-88.03790407006068
-101.15867778338773
-110.95519407191219
-110.33368791239823
-106.16467956841919
-93.28001600918478
-90.30766192430065
-90.221852

-169.3539137873113
-169.22519958525294
-169.09648538319456
-168.9677711811362
-168.8390569790778
-168.71034277701943
-168.58162857496106
-168.45291437290268
-168.3242001708443
-168.19548596878593
-168.06677176672756
-167.93805756466918
-167.8093433626108
-167.68062916055243
-167.55191495849405
-167.42320075643568
-167.2944865543773
-167.16577235231892
-167.03705815026055
-166.9083439482022
-114.47282878686147
-166.65091554408545
-138.30672208031447
-119.17485791341798
-121.97764835032822
-118.40319606733485
-152.18370737543142
-152.0630378110017
-139.7592377614795
-128.82252660634867
-165.49248772556007
-106.19594770012313
-151.45968998885309
-165.10634511938494
-164.97763091732656
-164.8489167152682
-119.00028070353252
-164.59148831115144
-96.73713209274442
-109.45783243027603
-101.99418572893822
-102.37225048113865
-163.94791730085956
-88.03790407006068
-101.15867778338773
-110.95519407191219
-110.33368791239823
-106.16467956841919
-93.28001600918478
-90.30766192430065
-90.2218524562

-125.2985409652791
-121.53601476836806
-126.33727695466294
-130.08960799449974
-148.78445369513636
-162.25275973942044
-162.12404553736206
-161.9953313353037
-161.8666171332453
-161.73790293118694
-148.060436308558
-135.2918528809408
-161.3517603250118
-147.69842761526883
-147.5777580508391
-160.96561771883668
-160.8369035167783
-160.70818931471993
-160.57947511266156
-160.45076091060318
-160.3220467085448
-160.19333250648643
-160.06461830442805
-159.93590410236968
-159.8071899003113
-159.67847569825292
-159.54976149619458
-159.42104729413617
-159.29233309207783
-159.16361889001945
-159.03490468796107
-145.52637545553375
-145.40570589110402
-133.4935136015758
-122.90489823310628
-116.82553953906242
-120.25391943200884
-101.86332566151097
-105.21237839234661
-95.21549837842514
-96.91387333384094
-94.5933014125412
-89.82541199767763
-89.3271466421771
-89.6498926312344
-157.10419165708544
-136.9616729781377
-82.35408832624995
-101.78389003771136
-93.868576286582
-78.84380159691412
-76.675

-168.77105438893523
-168.64234018687685
-168.5136259848185
-168.38491178276013
-168.25619758070175
-168.12748337864338
-167.998769176585
-167.87005497452662
-167.74134077246825
-167.61262657040987
-167.4839123683515
-167.35519816629312
-167.22648396423475
-167.09776976217637
-166.969055560118
-152.96464189818087
-127.19830930630232
-125.85262996240235
-152.6026332048917
-166.32548454982611
-152.36129407603224
-135.14218754469624
-140.64629415673886
-117.58339970026782
-108.42378965891427
-102.02135130858815
-151.63727668945387
-165.2957709333591
-165.16705673130073
-49.483023677263674
-118.57689604777678
-164.78091412512563
-110.73333862118287
-106.07269915303405
-103.07464298663095
-110.44191023916389
-95.21191099302885
-102.32338634450923
-100.36815452831974
-126.63292896912824
-106.3996137341048
-98.72800426712624
-93.83090538805689
-90.43394580028345
-90.34813633224454
-89.06039978268319
-88.57929357862639
-90.90222501152508
-90.40928773156537
-95.35473512143503
-104.95640959706733

-169.3539137873113
-169.22519958525294
-169.09648538319456
-168.9677711811362
-168.8390569790778
-168.71034277701943
-168.58162857496106
-168.45291437290268
-168.3242001708443
-168.19548596878593
-168.06677176672756
-167.93805756466918
-167.8093433626108
-167.68062916055243
-167.55191495849405
-167.42320075643568
-167.2944865543773
-167.16577235231892
-167.03705815026055
-166.9083439482022
-114.47282878686147
-166.65091554408545
-138.30672208031447
-119.17485791341798
-121.97764835032822
-118.40319606733485
-152.18370737543142
-152.0630378110017
-139.7592377614795
-128.82252660634867
-165.49248772556007
-106.19594770012313
-151.45968998885309
-165.10634511938494
-164.97763091732656
-164.8489167152682
-119.00028070353252
-164.59148831115144
-96.73713209274442
-109.45783243027603
-101.99418572893822
-102.37225048113865
-163.94791730085956
-88.03790407006068
-101.15867778338773
-110.95519407191219
-110.33368791239823
-106.16467956841919
-93.28001600918478
-90.30766192430065
-90.2218524562

...

-168.83176600085105
-168.70305179879267
-168.5743375967343
-168.44562339467592
-168.31690919261754
-168.18819499055917
-168.0594807885008
-167.93076658644242
-167.80205238438404
-167.67333818232567
-167.5446239802673
-167.4159097782089
-167.28719557615054
-167.15848137409216
-140.17035190948025
-120.14907339734899
-129.78180730659753
-152.7802199054925
-141.88315921226692
-152.53888077663305
-138.78097361432137
-135.19441258720445
-115.50766078243092
-108.06030521489853
-151.93553295448442
-165.61391094939168
-165.4851967473333
-48.40363103887667
-71.3738112496937
-118.72644274301959
-97.52824128723688
-114.06951595157028
-102.35781498028632
-102.26560182060271
-106.5172480104537
-93.2219064589124
-101.52122480482112
-102.36701145486788
-107.13394727602432
-106.04345340164988
-93.6251032590477
-93.53759218682671
-90.56022967626623
-90.47442020822731
-86.83837999042768
-91.9373366797017
-93.10003682572176
-91.35220781788732
-162.65348430204907
-106.08516488888199
-105.49394204770385
-10

-76.24803076277692
-75.82534771783739
-154.29435214648146
-76.00606150564369
-82.01457540482929
-153.90820954030633
-67.74111784602819
-153.65078113618958
-169.54333960128548
-169.4146253992271
-169.28591119716873
-169.15719699511035
-169.02848279305198
-168.8997685909936
-168.77105438893523
-168.64234018687685
-168.5136259848185
-168.38491178276013
-168.25619758070175
-168.12748337864338
-167.998769176585
-167.87005497452662
-167.74134077246825
-167.61262657040987
-167.4839123683515
-167.35519816629312
-167.22648396423475
-167.09776976217637
-166.969055560118
-152.96464189818087
-127.19830930630232
-125.85262996240235
-152.6026332048917
-166.32548454982611
-152.36129407603224
-135.14218754469624
-140.64629415673886
-117.58339970026782
-108.42378965891427
-102.02135130858815
-151.63727668945387
-165.2957709333591
-165.16705673130073
-49.483023677263674
-118.57689604777678
-164.78091412512563
-110.73333862118287
-106.07269915303405
-103.07464298663095
-110.44191023916389
-95.21191099302

-169.3539137873113
-169.22519958525294
-169.09648538319456
-168.9677711811362
-168.8390569790778
-168.71034277701943
-168.58162857496106
-168.45291437290268
-168.3242001708443
-168.19548596878593
-168.06677176672756
-167.93805756466918
-167.8093433626108
-167.68062916055243
-167.55191495849405
-167.42320075643568
-167.2944865543773
-167.16577235231892
-167.03705815026055
-166.9083439482022
-114.47282878686147
-166.65091554408545
-138.30672208031447
-119.17485791341798
-121.97764835032822
-118.40319606733485
-152.18370737543142
-152.0630378110017
-139.7592377614795
-128.82252660634867
-165.49248772556007
-106.19594770012313
-151.45968998885309
-165.10634511938494
-164.97763091732656
-164.8489167152682
-119.00028070353252
-164.59148831115144
-96.73713209274442
-109.45783243027603
-101.99418572893822
-102.37225048113865
-163.94791730085956
-88.03790407006068
-101.15867778338773
-110.95519407191219
-110.33368791239823
-106.16467956841919
-93.28001600918478
-90.30766192430065
-90.2218524562

-148.35869257358854
-126.90142793826806
-134.07198666127505
-161.54118613898598
-147.87601431586964
-147.7553447514399
-161.15504353281085
-161.02632933075247
-160.89761512869413
-160.76890092663575
-160.64018672457738
-160.511472522519
-160.38275832046062
-160.25404411840225
-160.12532991634387
-159.9966157142855
-159.86790151222712
-159.73918731016875
-159.61047310811037
-159.481758906052
-159.35304470399362
-159.22433050193524
-145.70396215613457
-138.03614437397155
-132.96896353473417
-121.22529325253751
-115.83043759331468
-112.92585449016525
-104.45096763553391
-103.85931165376516
-96.68915191103372
-97.04917748667964
-91.71144260640901
-91.20206237402965
-89.86680627852931
-90.60733171870947
-97.49822842231828
-157.16490326900126
-104.98540589957955
-101.44083605406857
-84.19327755911851
-78.25963595953871
-77.48221460075783
-78.09747633489823
-77.66773880916816
-75.87165521087397
-76.4721580730239
-86.71056932666721
-82.01282287568074
-71.30385970658044
-70.59778799536855
-101.

-133.4621771820118
-130.73501219165743
-143.75107157138785
-153.73190583675515
-145.7622856605839
-142.67108584156034
-130.19870301641419
-130.73695439105677
-119.6640629565222
-166.8864710135219
-152.88721888574705
-152.7665493213173
-166.50032840734679
-50.745865954719534
-49.023855883872955
-51.472540747552586
-110.69639119933498
-115.94729766223602
-165.72804319499656
-105.38928017668226
-105.29566984791254
-107.6693350502066
-107.57428456253271
-101.68753968643657
-102.5317945248809
-101.97050270041993
-99.5757866535105
-94.65219457486668
-92.46730205002538
-91.15117464823632
-89.0621293558111
-90.17209490711892
-92.95099407854374
-93.70270025322665
-99.30406714969118
-103.89151491636662
-105.25633352876756
-111.79984301086935
-110.12222948554879
-117.0890670671873
-125.2985409652791
-121.53601476836806
-126.33727695466294
-130.08960799449974
-148.78445369513636
-162.25275973942044
-162.12404553736206
-161.9953313353037
-161.8666171332453
-161.73790293118694
-148.060436308558
-135